# Sandy Ouflow Analysis
## Organising the Dataset
We import ```pandas```, ```numpy```, ```csv```, and ```os``` libraries

In [1]:
import pandas as pd
import numpy as np
import csv
import os

We store the path to the ```IRS_migration_data``` repository folder in a string variable

In [2]:
repo_path = os.getcwd()[0:len(os.getcwd())-7]

We also create a new ```tables``` folder in which we will store the tables produced by this script, it will be a subfolder of your ```IRS_migration_data``` repository

In [33]:
results_path = repo_path + 'tables/'
if not os.path.exists(results_path):
    os.makedirs(results_path)

We upload the data on outflows from a csv file.
It covers the period 1998-2015.

In [3]:
outflow_df = pd.read_csv(repo_path + 'outflows/outflow.csv')

We print the first 10 lines of the dataframe we just created to see how it is structured

In [4]:
outflow_df.head()

,year,state_code_origin,county_code_origin,origin,state_code_dest,county_code_dest,destination,state_abbrv,county_name,return_num,exmpt_num,aggr_agi
0,1998-1999,0,0,0,96,0,96000,US,Total Mig - US & For,6826974,13018600,272074430
1,1998-1999,0,0,0,97,0,97000,US,Total Mig - US,6711480,12800546,268111409
2,1998-1999,0,0,0,97,1,97001,US,Total Mig - US Same St,3803650,7221528,143410291
3,1998-1999,0,0,0,97,3,97003,US,Total Mig - US Diff St,2907830,5579018,124701118
4,1998-1999,0,0,0,98,0,98000,US,Total Mig - Foreign,115494,218054,3963021


We store in a numpy array the unique fip codes of destination and origin counties

In [5]:
origins = outflow_df[(outflow_df['state_code_origin']<=56) & ((outflow_df['state_code_dest']<=56))]
origin_codes = pd.unique(origins['origin'].values)
destination_codes = np.append(origin_codes, [58000,59000,57009])

We upload the cleaned and restructured datasets for the six period we will analyse:
    
* before Sandy (2010-2011);
* and after Sandy (2012-2013).

In [7]:
pre_1 = pd.read_csv(repo_path + 'outflows/1011out.csv')
pre_1.rename(columns={'Unnamed: 0':''}, inplace=True)
pre_1.set_index([''], inplace=True)
new_col_names = list(map(int, pre_1.columns.values))
pre_1.columns = new_col_names

pre_2 = pd.read_csv(repo_path + 'outflows/1112out.csv')
pre_2.rename(columns={'Unnamed: 0':''}, inplace=True)
pre_2.set_index([''], inplace=True)
new_col_names = list(map(int, pre_2.columns.values))
pre_2.columns = new_col_names

re_1 = pd.read_csv(repo_path + 'outflows/1213out.csv')
re_1.rename(columns={'Unnamed: 0':''}, inplace=True)
re_1.set_index([''], inplace=True)
new_col_names = list(map(int, re_1.columns.values))
re_1.columns = new_col_names

re_2 = pd.read_csv(repo_path + 'outflows/1314out.csv')
re_2.rename(columns={'Unnamed: 0':''}, inplace=True)
re_2.set_index([''], inplace=True)
new_col_names = list(map(int, re_2.columns.values))
re_2.columns = new_col_names

We adjust the estimates for all the years after 2011 in order to account for a change in methodology by the IRS. Since improvements in the collection mechanisms allowed the IRS to increase by 4.7 percent the coverage rate, we will decrease by 4.7 percent all the flows in order to make them comparable to their pre 2011 equivalents.

In [8]:
pre_2_adj = pre_2 * 0.963
re_1_adj = re_1 * 0.963
re_2_adj = re_2 * 0.963

We import a set of csv files that contain the fip codes for different groups of counties we will use in the analysis.

In [9]:
disaster_sandy_counties_df = pd.read_csv(repo_path + 'county_groups/disaster_sandy_counties.csv', usecols = ['fip_code'])
nearby_sandy_counties_df = pd.read_csv(repo_path + 'county_groups/nearby_sandy_counties.csv', usecols = ['fip_code'])
distant_sandy_counties_df = pd.read_csv(repo_path + 'county_groups/distant_sandy_counties.csv', usecols = ['fip_code'])
urban_nc_counties_df = pd.read_csv(repo_path + 'county_groups/urban_nc_counties.csv', usecols = ['fip_code'])

We now convert the dataframes into lists and we add one list with all the counties

In [10]:
disaster_sandy_counties = list(disaster_sandy_counties_df['fip_code'])
nearby_sandy_counties = list(nearby_sandy_counties_df['fip_code'])
distant_sandy_counties = list(distant_sandy_counties_df['fip_code'])
urban_nc_counties = list(urban_nc_counties_df['fip_code'])
all_nc_counties = disaster_sandy_counties + nearby_sandy_counties + distant_sandy_counties

Finally, using list comprehension, we divide all the groups we have defined so far into urban and rural areas by looking at their 2010 Census population. If the proportion living in rural areas is equal or above 50% we classify the county as rural otherwise as urban.

In [11]:
disaster_sandy_urban_counties = [x for x in disaster_sandy_counties if x in urban_nc_counties]
nearby_sandy_urban_counties = [x for x in nearby_sandy_counties if x in urban_nc_counties]
distant_sandy_urban_counties = [x for x in distant_sandy_counties if x in urban_nc_counties]

We now summarize the number of counties in each group:

In [12]:
print('There are', len(disaster_sandy_counties), 'disaster counties, of which', 
      len(disaster_sandy_urban_counties), 'are urban.' )

print('There are', len(nearby_sandy_counties), 'nearby counties, of which', 
      len(nearby_sandy_urban_counties), 'are urban.' )

print('There are', len(distant_sandy_counties), 'distant counties, of which', 
      len(distant_sandy_urban_counties), 'are urban.' )

print('There is a total of', len(all_nc_counties), 'counties, of which', 
      len(urban_nc_counties), 'are urban and the remaining',
      len(all_nc_counties) - len(urban_nc_counties), 'are rural.' )

There are 41 disaster counties, of which 40 are urban.
There are 132 nearby counties, of which 84 are urban.
There are 2939 distant counties, of which 1123 are urban.
There is a total of 3112 counties, of which 1247 are urban and the remaining 1865 are rural.


## Ties Analysis

We create six dataframes, one for each year considered, where we have the ties connecting each county to the others. Here a tie is defined as the presence of a flow of any size between two counties. Given that there has been a change in methodology in the IRS data collection such that before 2012 flows up to 10 households were recorded while after only flows up to 20 households are recoded, we need to use this latter threshold in order not to bias the results.

In [13]:
pre_1_ties = pre_1.drop([58000,59000,57009],axis=1).where(pre_1<20,1)
pre_1_ties = pre_1_ties.where(pre_1_ties==1,0)

pre_2_ties = pre_2.drop([58000,59000,57009],axis=1).where(pre_2<20,1)
pre_2_ties = pre_2_ties.where(pre_2_ties==1,0)

re_1_ties = re_1.drop([58000,59000,57009],axis=1).where(re_1<20,1)
re_1_ties = re_1_ties.where(re_1_ties==1,0)

re_2_ties = re_2.drop([58000,59000,57009],axis=1).where(re_2<20,1)
re_2_ties = re_2_ties.where(re_2_ties==1,0)

The final result is a matrix for each period whose rows and columns are all the counties in the dataset and where a 1 indicates the presence of a tie between the two counties and a 0 its absence. We consider a tie to exist if a positive flow was recorded at least in one of the three years composing the before and after periods.

In [14]:
pre_ties = (pre_1_ties + pre_2_ties)
pre_ties = pre_ties.where(pre_ties==0,1)
re_ties = (re_1_ties + re_2_ties)
re_ties = re_ties.where(re_ties==0,1)

Finally, we compute the number of unique ties in the before and recovery periods. A tie is unique if present only in one of the two periods.

In [15]:
ties = re_ties - pre_ties
uties_pre = ties.where(ties==-1,0)*-1
uties_re = ties.where(ties==1,0)

In [16]:
for i in uties_pre.index:
    uties_pre.loc[i, i] = 0
for i in uties_re.index:
    uties_re.loc[i, i] = 0

We set up the table headers

In [17]:
county_groups= ['All','Disaster Affected','Nearby','Distant',
                   'All (Urban)', 'Disaster Affected (Urban)', 'Nearby (Urban)','Distant (Urban)']
periods = ['Pre-Disaster','Recovery','% Change']

We initialise the table

In [18]:
ties_df = pd.DataFrame(0, index=county_groups, columns=periods)

Finally we fill it

In [41]:
ties_df.loc['All',
            'Pre-Disaster'] = uties_pre.loc[disaster_sandy_counties,
                                            all_nc_counties].sum(axis=1).sum(axis=0)
ties_df.loc['All',
            'Recovery'] = uties_re.loc[disaster_sandy_counties,
                                       all_nc_counties].sum(axis=1).sum(axis=0)

ties_df.loc['Disaster Affected',
            'Pre-Disaster'] = uties_pre.loc[disaster_sandy_counties,
                                            disaster_sandy_counties].sum(axis=1).sum(axis=0)
ties_df.loc['Disaster Affected',
            'Recovery'] = uties_re.loc[disaster_sandy_counties,
                                       disaster_sandy_counties].sum(axis=1).sum(axis=0)

ties_df.loc['Nearby',
            'Pre-Disaster'] = uties_pre.loc[disaster_sandy_counties,
                                            nearby_sandy_counties].sum(axis=1).sum(axis=0)
ties_df.loc['Nearby',
            'Recovery'] = uties_re.loc[disaster_sandy_counties,
                                       nearby_sandy_counties].sum(axis=1).sum(axis=0)

ties_df.loc['Distant',
            'Pre-Disaster'] = uties_pre.loc[disaster_sandy_counties,
                                            distant_sandy_counties].sum(axis=1).sum(axis=0)
ties_df.loc['Distant',
            'Recovery'] = uties_re.loc[disaster_sandy_counties,
                                       distant_sandy_counties].sum(axis=1).sum(axis=0)

ties_df.loc['All (Urban)',
            'Pre-Disaster'] = uties_pre.loc[disaster_sandy_counties,
                                            urban_nc_counties].sum(axis=1).sum(axis=0)
ties_df.loc['All (Urban)',
            'Recovery'] = uties_re.loc[disaster_sandy_counties,
                                       urban_nc_counties].sum(axis=1).sum(axis=0)

ties_df.loc['Disaster Affected (Urban)',
            'Pre-Disaster'] = uties_pre.loc[disaster_sandy_counties,
                                            disaster_sandy_urban_counties].sum(axis=1).sum(axis=0)
ties_df.loc['Disaster Affected (Urban)',
            'Recovery'] = uties_re.loc[disaster_sandy_counties,
                                       disaster_sandy_urban_counties].sum(axis=1).sum(axis=0)

ties_df.loc['Nearby (Urban)',
            'Pre-Disaster'] = uties_pre.loc[disaster_sandy_counties,
                                            nearby_sandy_urban_counties].sum(axis=1).sum(axis=0)
ties_df.loc['Nearby (Urban)',
            'Recovery'] = uties_re.loc[disaster_sandy_counties,
                                       nearby_sandy_urban_counties].sum(axis=1).sum(axis=0)

ties_df.loc['Distant (Urban)',
            'Pre-Disaster'] = uties_pre.loc[disaster_sandy_counties,
                                            distant_sandy_urban_counties].sum(axis=1).sum(axis=0)
ties_df.loc['Distant (Urban)',
            'Recovery'] = uties_re.loc[disaster_sandy_counties,
                                       distant_sandy_urban_counties].sum(axis=1).sum(axis=0)

ties_df.loc[:,'% Change'] = (ties_df.loc[:,'Recovery'] - ties_df.loc[:,'Pre-Disaster'])/ties_df.loc[:,'Pre-Disaster']*100

ties_df.loc[:,'% Change'] = ties_df.loc[:,'% Change'].round(decimals=1)
ties_df

,Pre-Disaster,Recovery,% Change
All,265.0,420.0,58.5
Disaster Affected,30.0,37.0,23.3
Nearby,71.0,94.0,32.4
Distant,164.0,289.0,76.2
All (Urban),258.0,412.0,59.7
Disaster Affected (Urban),29.0,37.0,27.6
Nearby (Urban),68.0,90.0,32.4
Distant (Urban),161.0,285.0,77.0


To export the table to a csv file, uncomment the following line

In [34]:
#ties_df.to_csv(results_path + 'outties_table_sandy.csv')

## Flows Analysis
We apply the same adjustment to flows, removing all flows below 20 households

In [20]:
pre_1_flows = pre_1.where(pre_1>=20,0)
pre_2_flows = pre_2.where(pre_2>=20,0)
re_1_flows = re_1.where(re_1>=20,0)
re_2_flows = re_2.where(re_2>=20,0)

We create to dataframes with the same structure as the ones with the ties but containing average flows for the two periods respectively

In [21]:
pre_avg = (pre_1_flows + pre_2_flows)/2
re_avg = (re_1_flows + re_2_flows)/2

pre_avg = pre_avg.round(decimals=0)
re_avg = re_avg.round(decimals=0)

pre_flows = pre_avg
re_flows = re_avg

We remove flows in the main diagonal as these represent household that remained in the same counties and are thus not interesting in our migration analysis 

In [22]:
for i in pre_flows.index:
    pre_flows.loc[i, i] = 0
for i in re_flows.index:
    re_flows.loc[i, i] = 0

We initialise the flows table as before

In [23]:
flows_df = pd.DataFrame(0, index=county_groups, columns=periods)

and now we fill it

In [24]:
flows_df.loc['All',
            'Pre-Disaster'] = pre_avg.loc[disaster_sandy_counties,
                                         all_nc_counties].sum(axis=1).sum(axis=0)
flows_df.loc['All',
            'Recovery'] = re_avg.loc[disaster_sandy_counties,
                                    all_nc_counties].sum(axis=1).sum(axis=0)

flows_df.loc['Disaster Affected',
            'Pre-Disaster'] = pre_avg.loc[disaster_sandy_counties, 
                                          disaster_sandy_counties].sum(axis=1).sum(axis=0)
flows_df.loc['Disaster Affected',
            'Recovery'] = re_avg.loc[disaster_sandy_counties, 
                                     disaster_sandy_counties].sum(axis=1).sum(axis=0)

flows_df.loc['Nearby',
            'Pre-Disaster'] = pre_avg.loc[disaster_sandy_counties, 
                                          nearby_sandy_counties].sum(axis=1).sum(axis=0)
flows_df.loc['Nearby',
            'Recovery'] = re_avg.loc[disaster_sandy_counties, 
                                     nearby_sandy_counties].sum(axis=1).sum(axis=0)

flows_df.loc['Distant',
            'Pre-Disaster'] = pre_avg.loc[disaster_sandy_counties, 
                                          distant_sandy_counties].sum(axis=1).sum(axis=0)
flows_df.loc['Distant',
            'Recovery'] = re_avg.loc[disaster_sandy_counties, 
                                     distant_sandy_counties].sum(axis=1).sum(axis=0)

flows_df.loc['All (Urban)',
            'Pre-Disaster'] = pre_avg.loc[disaster_sandy_counties, 
                                          urban_nc_counties].sum(axis=1).sum(axis=0)
flows_df.loc['All (Urban)',
            'Recovery'] = re_avg.loc[disaster_sandy_counties, 
                                     urban_nc_counties].sum(axis=1).sum(axis=0)

flows_df.loc['Disaster Affected (Urban)',
            'Pre-Disaster'] = pre_avg.loc[disaster_sandy_counties, 
                                          disaster_sandy_urban_counties].sum(axis=1).sum(axis=0)
flows_df.loc['Disaster Affected (Urban)',
            'Recovery'] = re_avg.loc[disaster_sandy_counties, 
                                     disaster_sandy_urban_counties].sum(axis=1).sum(axis=0)

flows_df.loc['Nearby (Urban)',
            'Pre-Disaster'] = pre_avg.loc[disaster_sandy_counties, 
                                          nearby_sandy_urban_counties].sum(axis=1).sum(axis=0)
flows_df.loc['Nearby (Urban)',
            'Recovery'] = re_avg.loc[disaster_sandy_counties, 
                                     nearby_sandy_urban_counties].sum(axis=1).sum(axis=0)

flows_df.loc['Distant (Urban)',
            'Pre-Disaster'] = pre_avg.loc[disaster_sandy_counties, 
                                          distant_sandy_urban_counties].sum(axis=1).sum(axis=0)
flows_df.loc['Distant (Urban)',
            'Recovery'] = re_avg.loc[disaster_sandy_counties, 
                                     distant_sandy_urban_counties].sum(axis=1).sum(axis=0)

flows_df.loc[:,'% Change'] = (flows_df.loc[:,'Recovery'] - flows_df.loc[:,'Pre-Disaster'])/flows_df.loc[:,'Pre-Disaster']*100
flows_df.loc[:,'% Change'] = flows_df.loc[:,'% Change'].round(decimals=1)
flows_df

,Pre-Disaster,Recovery,% Change
All,474131.0,506697.0,6.9
Disaster Affected,325889.0,341816.0,4.9
Nearby,56717.0,58540.0,3.2
Distant,91525.0,106341.0,16.2
All (Urban),471649.0,503973.0,6.9
Disaster Affected (Urban),324970.0,340879.0,4.9
Nearby (Urban),55296.0,56949.0,3.0
Distant (Urban),91383.0,106145.0,16.2


To export the table to a csv file, uncomment the following line

In [25]:
#flows_df.to_csv(results_path + outflows_table_sandy.csv')

## Adjusted Data From 2011
We now repeat the analysis this time using the adjusted data

In [26]:
pre_2_ties_adj = pre_2_adj.drop([58000,59000,57009],axis=1).where(pre_2_adj<20,1)
pre_2_ties_adj = pre_2_ties_adj.where(pre_2_ties_adj==1,0)

re_1_ties_adj = re_1_adj.drop([58000,59000,57009],axis=1).where(re_1_adj<20,1)
re_1_ties_adj = re_1_ties_adj.where(re_1_ties_adj==1,0)

re_2_ties_adj = re_2_adj.drop([58000,59000,57009],axis=1).where(re_2_adj<20,1)
re_2_ties_adj = re_2_ties_adj.where(re_2_ties_adj==1,0)

The final result is a matrix for each period whose rows and columns are all the counties in the dataset and where a 1 indicates the presence of a tie between the two counties and a 0 its absence. We consider a tie to exist if a positive flow was recorded at least in one of the three years composing the before and after periods.

In [27]:
pre_ties_adj = (pre_1_ties + pre_2_ties_adj)
pre_ties_adj = pre_ties_adj.where(pre_ties_adj==0,1)
re_ties_adj = (re_1_ties_adj + re_2_ties_adj)
re_ties_adj = re_ties_adj.where(re_ties_adj==0,1)

Finally, we compute the number of unique ties in the before and recovery periods. A tie is unique if present only in one of the two periods.

In [28]:
ties_adj = re_ties_adj - pre_ties_adj
uties_pre_adj = ties_adj.where(ties_adj==-1,0)*-1
uties_re_adj = ties_adj.where(ties_adj==1,0)

In [29]:
for i in uties_pre_adj.index:
    uties_pre_adj.loc[i, i] = 0
for i in uties_re_adj.index:
    uties_re_adj.loc[i, i] = 0

We initialise the table

In [30]:
ties_adj_df = pd.DataFrame(0, index=county_groups, columns=periods)

Finally we fill it

In [31]:
ties_adj_df.loc['All',
            'Pre-Disaster'] = uties_pre_adj.loc[disaster_sandy_counties,
                                            all_nc_counties].sum(axis=1).sum(axis=0)
ties_adj_df.loc['All',
            'Recovery'] = uties_re_adj.loc[disaster_sandy_counties,
                                       all_nc_counties].sum(axis=1).sum(axis=0)

ties_adj_df.loc['Disaster Affected',
            'Pre-Disaster'] = uties_pre_adj.loc[disaster_sandy_counties,
                                            disaster_sandy_counties].sum(axis=1).sum(axis=0)
ties_adj_df.loc['Disaster Affected',
            'Recovery'] = uties_re_adj.loc[disaster_sandy_counties,
                                       disaster_sandy_counties].sum(axis=1).sum(axis=0)

ties_adj_df.loc['Nearby',
            'Pre-Disaster'] = uties_pre_adj.loc[disaster_sandy_counties,
                                            nearby_sandy_counties].sum(axis=1).sum(axis=0)
ties_adj_df.loc['Nearby',
            'Recovery'] = uties_re_adj.loc[disaster_sandy_counties,
                                       nearby_sandy_counties].sum(axis=1).sum(axis=0)

ties_adj_df.loc['Distant',
            'Pre-Disaster'] = uties_pre_adj.loc[disaster_sandy_counties,
                                            distant_sandy_counties].sum(axis=1).sum(axis=0)
ties_adj_df.loc['Distant',
            'Recovery'] = uties_re_adj.loc[disaster_sandy_counties,
                                       distant_sandy_counties].sum(axis=1).sum(axis=0)

ties_adj_df.loc['All (Urban)',
            'Pre-Disaster'] = uties_pre_adj.loc[disaster_sandy_counties,
                                            urban_nc_counties].sum(axis=1).sum(axis=0)
ties_adj_df.loc['All (Urban)',
            'Recovery'] = uties_re_adj.loc[disaster_sandy_counties,
                                       urban_nc_counties].sum(axis=1).sum(axis=0)

ties_adj_df.loc['Disaster Affected (Urban)',
            'Pre-Disaster'] = uties_pre_adj.loc[disaster_sandy_counties,
                                            disaster_sandy_urban_counties].sum(axis=1).sum(axis=0)
ties_adj_df.loc['Disaster Affected (Urban)',
            'Recovery'] = uties_re_adj.loc[disaster_sandy_counties,
                                       disaster_sandy_urban_counties].sum(axis=1).sum(axis=0)

ties_adj_df.loc['Nearby (Urban)',
            'Pre-Disaster'] = uties_pre_adj.loc[disaster_sandy_counties,
                                            nearby_sandy_urban_counties].sum(axis=1).sum(axis=0)
ties_adj_df.loc['Nearby (Urban)',
            'Recovery'] = uties_re_adj.loc[disaster_sandy_counties,
                                       nearby_sandy_urban_counties].sum(axis=1).sum(axis=0)

ties_adj_df.loc['Distant (Urban)',
            'Pre-Disaster'] = uties_pre_adj.loc[disaster_sandy_counties,
                                            distant_sandy_urban_counties].sum(axis=1).sum(axis=0)
ties_adj_df.loc['Distant (Urban)',
            'Recovery'] = uties_re_adj.loc[disaster_sandy_counties,
                                       distant_sandy_urban_counties].sum(axis=1).sum(axis=0)

ties_adj_df.loc[:,'% Change'] = (ties_adj_df.loc[:,'Recovery'] - ties_adj_df.loc[:,'Pre-Disaster'])/ties_adj_df.loc[:,'Pre-Disaster']*100

ties_adj_df.loc[:,'% Change'] = ties_adj_df.loc[:,'% Change'].round(decimals=1)
ties_adj_df

,Pre-Disaster,Recovery,% Change
All,254.0,378.0,48.8
Disaster Affected,29.0,33.0,13.8
Nearby,76.0,88.0,15.8
Distant,149.0,257.0,72.5
All (Urban),246.0,370.0,50.4
Disaster Affected (Urban),28.0,33.0,17.9
Nearby (Urban),71.0,84.0,18.3
Distant (Urban),147.0,253.0,72.1


To export the table to a csv file, uncomment the following line

In [35]:
#ties_adj_df.to_csv(results_path + 'outties_table_adj.csv')

## Flows Analysis
We apply the same adjustment to flows, removing all flows below 20 households

In [36]:
pre_2_flows_adj = pre_2_adj.where(pre_2_adj>=20,0)
re_1_flows_adj = re_1_adj.where(re_1_adj>=20,0)
re_2_flows_adj = re_2_adj.where(re_2_adj>=20,0)

We create to dataframes with the same structure as the ones with the ties but containing average flows for the two periods respectively

In [37]:
pre_avg_adj = (pre_2_flows + pre_2_flows_adj)/3
re_avg_adj = (re_1_flows_adj + re_2_flows_adj)/2

pre_avg_adj = pre_avg_adj.round(decimals=0)
re_avg_adj = re_avg_adj.round(decimals=0)

pre_flows_adj = pre_avg_adj
re_flows_adj = re_avg_adj

We remove flows in the main diagonal as these represent household that remained in the same counties and are thus not interesting in our migration analysis 

In [38]:
for i in pre_flows_adj.index:
    pre_flows_adj.loc[i, i] = 0
for i in re_flows_adj.index:
    re_flows_adj.loc[i, i] = 0

We initialise the flows table as before

In [39]:
flows_adj_df = pd.DataFrame(0, index=county_groups, columns=periods)

and now we fill it

In [48]:
flows_adj_df.loc['All',
            'Pre-Disaster'] = pre_flows_adj.loc[disaster_sandy_counties,
                                         all_nc_counties].sum(axis=1).sum(axis=0)
flows_adj_df.loc['All',
            'Recovery'] = re_flows_adj.loc[disaster_sandy_counties,
                                    all_nc_counties].sum(axis=1).sum(axis=0)

flows_adj_df.loc['Disaster Affected',
            'Pre-Disaster'] = pre_flows_adj.loc[disaster_sandy_counties, 
                                          disaster_sandy_counties].sum(axis=1).sum(axis=0)
flows_adj_df.loc['Disaster Affected',
            'Recovery'] = re_flows_adj.loc[disaster_sandy_counties, 
                                     disaster_sandy_counties].sum(axis=1).sum(axis=0)

flows_adj_df.loc['Nearby',
            'Pre-Disaster'] = pre_flows_adj.loc[disaster_sandy_counties, 
                                          nearby_sandy_counties].sum(axis=1).sum(axis=0)
flows_adj_df.loc['Nearby',
            'Recovery'] = re_flows_adj.loc[disaster_sandy_counties, 
                                     nearby_sandy_counties].sum(axis=1).sum(axis=0)

flows_adj_df.loc['Distant',
            'Pre-Disaster'] = pre_flows_adj.loc[disaster_sandy_counties, 
                                          distant_sandy_counties].sum(axis=1).sum(axis=0)
flows_adj_df.loc['Distant',
            'Recovery'] = re_flows_adj.loc[disaster_sandy_counties, 
                                     distant_sandy_counties].sum(axis=1).sum(axis=0)

flows_adj_df.loc['All (Urban)',
            'Pre-Disaster'] = pre_flows_adj.loc[disaster_sandy_counties, 
                                          urban_nc_counties].sum(axis=1).sum(axis=0)
flows_adj_df.loc['All (Urban)',
            'Recovery'] = re_flows_adj.loc[disaster_sandy_counties, 
                                     urban_nc_counties].sum(axis=1).sum(axis=0)

flows_adj_df.loc['Disaster Affected (Urban)',
            'Pre-Disaster'] = pre_flows_adj.loc[disaster_sandy_counties, 
                                          disaster_sandy_urban_counties].sum(axis=1).sum(axis=0)
flows_adj_df.loc['Disaster Affected (Urban)',
            'Recovery'] = re_flows_adj.loc[disaster_sandy_counties, 
                                     disaster_sandy_urban_counties].sum(axis=1).sum(axis=0)

flows_adj_df.loc['Nearby (Urban)',
            'Pre-Disaster'] = pre_flows_adj.loc[disaster_sandy_counties, 
                                          nearby_sandy_urban_counties].sum(axis=1).sum(axis=0)
flows_adj_df.loc['Nearby (Urban)',
            'Recovery'] = re_flows_adj.loc[disaster_sandy_counties, 
                                     nearby_sandy_urban_counties].sum(axis=1).sum(axis=0)

flows_adj_df.loc['Distant (Urban)',
            'Pre-Disaster'] = pre_flows_adj.loc[disaster_sandy_counties, 
                                          distant_sandy_urban_counties].sum(axis=1).sum(axis=0)
flows_adj_df.loc['Distant (Urban)',
            'Recovery'] = re_flows_adj.loc[disaster_sandy_counties, 
                                     distant_sandy_urban_counties].sum(axis=1).sum(axis=0)

flows_adj_df.loc[:,'% Change'] = (flows_adj_df.loc[:,'Recovery'] - flows_adj_df.loc[:,'Pre-Disaster'])/flows_adj_df.loc[:,'Pre-Disaster']*100
flows_adj_df.loc[:,'% Change'] = flows_adj_df.loc[:,'% Change'].round(decimals=1)
flows_adj_df

,Pre-Disaster,Recovery,% Change
All,490208.0,486037.0,-0.9
Disaster Affected (Coastline),334472.0,328916.0,-1.7
Nearby,57470.0,55895.0,-2.7
Distant,98266.0,101226.0,3.0
All (Urban),487602.0,483459.0,-0.8
Disaster Affected (Urban),333466.0,328014.0,-1.6
Nearby (Urban),56032.0,54400.0,-2.9
Distant (Urban),98104.0,101045.0,3.0


To export the table to a csv file, uncomment the following line

In [40]:
#flows_adj_df.to_csv(results_path + 'outflows_table_adj.csv')